In [ ]:
from src.features import preprocessing
import river

In [ ]:
DATA_PATH = "./datasets/traffic/"

list_of_datafiles = preprocessing.get_list_of_datafiles(DATA_PATH, sort = True)
df = preprocessing.load_data(DATA_PATH, list_of_datafiles)
train_df, test_df = preprocessing.make_train_test_datasets(df, split_point=20000)

In [ ]:
n_input_steps = 1
n_output_steps = 1
#target = '5->8' # SMAPE: ~3.5
#target = '8->5' # SMAPE: ~3.5
target = '5->12' # SMAPE: ~20.0
#target = '8->12' # SMAPE: ~7.5
X, y = preprocessing.split_sequence(sequence = df[target], n_input_steps=n_input_steps, n_output_steps=n_output_steps)
#X_test, y_test = preprocessing.split_sequence(sequence = test_df[target], n_input_steps=n_input_steps, n_output_steps=n_output_steps)

In [ ]:
X, y

In [ ]:
import numpy as np

In [ ]:
# model = river.compose.Pipeline(
#     river.preprocessing.StandardScaler(),
#     river.linear_model.LinearRegression(river.optim.SGD(lr=0.3))
#     )
# metric = river.metrics.SMAPE()

# predicted = list()
# for xi, yi in river.stream.iter_array(X, y):
#     # Test the current model on the new "unobserved" sample
#     yi_pred = model.predict_one(xi)

#     # Update the running metric with the prediction and ground truth value
#     metric.update(yi[0], yi_pred)
#     predicted.append(yi_pred)

#     # Train the model with the new sample
#     model.learn_one(xi, yi[0])


# print(f'ROC AUC: {metric}')

In [ ]:
models = [
river.linear_model.LinearRegression(optimizer=river.optim.SGD(lr=lr))
for lr in [0.0001, 0.001, 1e-05, 0.01]
]

model = (
river.preprocessing.StandardScaler() |
    river.model_selection.EpsilonGreedyRegressor(
    models,
    epsilon=0.1,
    decay=0.001,
    burn_in=100,
    seed=1
    )
)

metric = river.metrics.SMAPE()

predicted = list()
for xi, yi in river.stream.iter_array(X, y):
    # Test the current model on the new "unobserved" sample
    yi_pred = model.predict_one(xi)

    # Update the running metric with the prediction and ground truth value
    metric.update(yi[0], yi_pred)
    predicted.append(yi_pred)

    # Train the model with the new sample
    model.learn_one(xi, yi[0])


print(f'{metric}')
model['EpsilonGreedyRegressor'].best_model

In [ ]:
for i in range(len(predicted)):
    print(f"real: {y[i]} | pred: {predicted[i]}")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
figure = plt.figure(figsize = (20, 15))
ax = plt.subplot(111)
line = ax.plot(np.arange(0, len(y), 1), y, 'r', linewidth=5)
line2 = ax.plot(np.arange(0, len(predicted), 1), predicted, 'b', linewidth=1)

box = ax.get_position()
ax.set_position([box.x0, box.y0 + box.height * 0.1,
                    box.width, box.height * 0.9])

ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
            fancybox=True, shadow=True, ncol=2)

plt.show()